In [1]:
import numpy as np
import pandas as pd
import math
from itertools import combinations
import numpy as np
from sklearn.impute import SimpleImputer
from datetime import datetime, timedelta

In [2]:
student_counts = pd.read_csv("Data/student_counts.csv")
pair_counts = pd.read_csv("Data/module_pairs.csv")
assignments = pd.read_csv("Data/Assignments_with_clash.csv")

In [3]:
instructors = pd.read_csv("Data/instructors.csv")
instructors = instructors.drop(['Days and Times'], axis = 1)
instructors = instructors.drop_duplicates()

In [4]:
assignments["Start Date"] = pd.to_datetime(assignments["Start Date"])
assignments["Due Date"] = pd.to_datetime(assignments["Due Date"])
assignments = assignments.sort_values(by=["Due Date"], na_position="last")

In [5]:
# Stress Score calculation = [weight(level)*1] + [weight(i_or_g)*2] + [weight(type)*3] + [weight(gap)*4] + weightage/100
# then standardize it to range 0 to 10

In [6]:
# how important/stressful each of these factors are to students
# sum to 10?
# level: 1
# i_or_g: 2
# assignment_type: 3
# gap: 4

In [7]:
# how stressful each level within the factors are
weights = {
    "level": {
        "level_1k": 0.1,
        "level_2k": 0.2,
        "level_3k": 0.3,
        "level_4k": 0.4
    },
    "i_or_g": {
        "I": 0.3,
        "G": 0.4,
        "I&G": 0.3
    },
    "type": {
        "Presentation": 0.1,
        "Project": 0.25,
        "Participation": 0.05,
        "Quiz": 0.1,
        "Assignment": 0.15,
        "Exam": 0.35
    },
    "gap": {
        "One Week": 0.4,
        "Two Weeks": 0.3,
        "More Than Two Weeks": 0.15,
        "Others": 0.15 # assume weight to be 0.15 if gap is not known eg. start date is NA
    }
}

# Individual Stress Score Calculation (Per Assignment)

### Assumptions & Helper Functions

In [8]:
# max_weightage = by_date["Weightage"].max()
# max weightage is 70 based on the data
# we will assume that weightage for any graded components will not exceed 70% based on department's guidelines

max_weightage = 70

# max_stress is assuming that this is the maximum possible stress for a student,so we use the highest weights for each factor
# Stress Score calculation = [weight(level)*1] + [weight(i_or_g)*2] + [weight(type)*3] + [weight(gap)*4] + weightage/100
max_stress = max_weightage/100 + 0.4*1 + 0.4*2 + 0.35*3 + 0.4*4

def normalized_score(score):
    return (score)/(max_stress)*10

In [9]:
def get_gap(due_date, start_date):
    if pd.isnull(start_date):
        return "Others"
    else:
        gap = (due_date - start_date).days
        if gap <= 7:
            return "One Week"
        elif gap <= 14:
            return "Two Weeks"
        else:
            return "More Than Two Weeks"

### Missing Value Imputation

In [10]:
# Impute missing values with strategy='most_frequent' for non-numeric columns and strategy='mean' for numeric columns
def impute(df):

    non_numeric_cols = df.select_dtypes(exclude='number').columns

    imputer = SimpleImputer(strategy='most_frequent')
    df.loc[:, non_numeric_cols] = imputer.fit_transform(df.loc[:, non_numeric_cols])

    imputer = SimpleImputer(strategy='mean')
    df.loc[:, ~df.columns.isin(non_numeric_cols)] = imputer.fit_transform(df.loc[:, ~df.columns.isin(non_numeric_cols)])
    return df

imputed_df = impute(assignments)
imputed_df

,Module Code,Semester,Assignment Name,Weightage,Assignment Type,Group or Individual,Start Date,Due Date,Deadline Time,Level,Major
4,DSA2101,2020.0,Quiz 1,15.986239,Quiz,I,2021-01-12,2021-01-19,23:59,level_2k,DSA
5,DSA2101,2020.0,Assignment 1,2.500000,Assignment,I,2021-01-19,2021-01-26,23:59,level_2k,DSA
6,DSA2101,2020.0,Assignment 2,2.500000,Assignment,I,2021-02-09,2021-02-16,23:59,level_2k,DSA
9,DSA2101,2020.0,Take-Home Midterms,30.000000,Assignment,I,2021-03-02,2021-03-06,23:59,level_2k,DSA
7,DSA2101,2020.0,Assignment 3,2.500000,Assignment,I,2022-09-28,2021-03-23,23:59,level_2k,DSA
...,...,...,...,...,...,...,...,...,...,...,...
88,ST4253,2210.0,Participation,10.000000,Participation,I,2022-09-28,2023-04-14,23:59,level_4k,ST
89,ST4253,2210.0,Tutorial Presentation,10.000000,Presentation,I,2022-09-28,2023-04-14,23:59,level_4k,ST
102,ST4248,2220.0,Presentation,10.000000,Presentation,G,2022-09-28,2023-04-14,23:59,level_4k,ST
104,ST3247,2220.0,Tutorial Attendance,10.000000,Participation,I,2022-09-28,2023-04-14,23:59,level_3k,ST


### Single function that takes in assignment details, outputs a stress score

In [11]:
def indiv_score(weightage, assignment_type, i_g, level, start_date, due_date):
    gap = get_gap(due_date, start_date)
    stress_score = weightage/100 + weights['level'][level] * 1 + weights['i_or_g'][i_g] * 2 + weights['type'][assignment_type] * 3 + weights['gap'][gap] * 4
    normalized_stress = normalized_score(stress_score)
    return normalized_stress

In [12]:
example2 = indiv_score(7.5, "Assignment", "I", "level_3k", None, None)
example2

4.45054945054945

### Single function that takes in a df, outputs a df with stress score column

In [13]:
def indiv_score_df(df):
    for i in range(0, len(df)):
        weightage = df.loc[i,"Weightage"]
        assignment_type = df.loc[i,"Assignment Type"]
        i_g = df.loc[i,"Group or Individual"]
        level = df.loc[i,"Level"]
        start_date = df.loc[i,"Start Date"]
        due_date = df.loc[i,"Due Date"]
        
        stress_score = indiv_score(weightage, assignment_type, i_g, level, start_date, due_date)
        df.loc[i, "Stress"] = stress_score
    return df

In [14]:
indiv_scores = indiv_score_df(assignments)
indiv_scores

,Module Code,Semester,Assignment Name,Weightage,Assignment Type,Group or Individual,Start Date,Due Date,Deadline Time,Level,Major,Stress
4,DSA2101,2020.0,Quiz 1,15.986239,Quiz,I,2021-01-12,2021-01-19,23:59,level_2k,DSA,6.285412
5,DSA2101,2020.0,Assignment 1,2.500000,Assignment,I,2021-01-19,2021-01-26,23:59,level_2k,DSA,6.318681
6,DSA2101,2020.0,Assignment 2,2.500000,Assignment,I,2021-02-09,2021-02-16,23:59,level_2k,DSA,6.318681
9,DSA2101,2020.0,Take-Home Midterms,30.000000,Assignment,I,2021-03-02,2021-03-06,23:59,level_2k,DSA,6.923077
7,DSA2101,2020.0,Assignment 3,2.500000,Assignment,I,2022-09-28,2021-03-23,23:59,level_2k,DSA,6.318681
...,...,...,...,...,...,...,...,...,...,...,...,...
88,ST4253,2210.0,Participation,10.000000,Participation,I,2022-09-28,2023-04-14,23:59,level_4k,ST,4.065934
89,ST4253,2210.0,Tutorial Presentation,10.000000,Presentation,I,2022-09-28,2023-04-14,23:59,level_4k,ST,4.395604
102,ST4248,2220.0,Presentation,10.000000,Presentation,G,2022-09-28,2023-04-14,23:59,level_4k,ST,4.835165
104,ST3247,2220.0,Tutorial Attendance,10.000000,Participation,I,2022-09-28,2023-04-14,23:59,level_3k,ST,3.846154


# Pairwise Stress Score Calculations for Clashes

## For >= 2 clashes

In [15]:
def get_pairwise_stress(pair_cnt, m1_cnt, m2_cnt, stress_sum):
    '''index = data[data['Due Date'] == date].index.values[0]
    stress_list = data.loc[index,"Stress"]
    if (data.loc[index, 'Module 1'] == module):
        if (math.isnan(data.loc[index, 'Module 1 Count']) | math.isnan(data.loc[index, 'Pair Count'])):
            pairwise_stress = data['Pair Count'].mean()/data['Module 1 Count'].mean()*sum(stress_list)+stress_list[0]
        else:
            pairwise_stress = data.loc[index, 'Pair Count']/data.loc[index, 'Module 1 Count']*sum(stress_list)+stress_list[0]
    else:
        if (math.isnan(data.loc[index, 'Module 2 Count']) | math.isnan(data.loc[index, 'Pair Count'])):
            pairwise_stress = data['Pair Count'].mean()/data['Module 2 Count'].mean()*sum(stress_list)+stress_list[1]
        else:
            pairwise_stress = data.loc[index, 'Pair Count']/data.loc[index, 'Module 2 Count']*sum(stress_list)+stress_list[1]
    return pairwise_stress'''
    
    # simplified pairwise_stress formula : (pair_cnt/m1_cnt + pair_cnt/m2_cnt + 1) * stress_sum/2
    pairwise_stress = (pair_cnt/m1_cnt + pair_cnt/m2_cnt + 1) * stress_sum/2
    
    return pairwise_stress

In [16]:
# returns all clash pairs on the specified date
def clash_pairs(df, date):
    data = []
    idx = list(df.index)
    assignment_combinations = list(combinations(idx, 2))
    
    # if no clashes
    if not assignment_combinations:
        return None
    else:
        for comb in assignment_combinations:
            assignment1 = comb[0]
            assignment2 = comb[1]
            scores = df.loc[df.index.isin([assignment1, assignment2]),'Stress'].tolist()
            modules = df.loc[df.index.isin([assignment1, assignment2]),'Module Code'].tolist()
            semester1 = df.loc[assignment1, "Semester"]
            semester2 = df.loc[assignment2, "Semester"]
            data.append({'Due Date': date, 'Semester': semester1, 'Module Code': modules, 'Assignment Index': [assignment1, assignment2], 'Stress': scores})

        df1 = pd.DataFrame(data)
        df1[['Module 1', 'Module 2']] = pd.DataFrame(df1['Module Code'].tolist(), index=df1.index)
        df1.drop('Module Code', axis=1, inplace=True)
    
        return df1

# Daily Stress Score Calculation

In [17]:
instructor_scores = indiv_scores.merge(instructors, how='left', on = ['Module Code', 'Semester'])
instructor_scores

,Module Code,Semester,Assignment Name,Weightage,Assignment Type,Group or Individual,Start Date,Due Date,Deadline Time,Level,Major,Stress,Instructor
0,DSA2101,2020.0,Quiz 1,15.986239,Quiz,I,2021-01-12,2021-01-19,23:59,level_2k,DSA,6.285412,s/o Gopal Vikneswaran
1,DSA2101,2020.0,Assignment 1,2.500000,Assignment,I,2021-01-19,2021-01-26,23:59,level_2k,DSA,6.318681,s/o Gopal Vikneswaran
2,DSA2101,2020.0,Assignment 2,2.500000,Assignment,I,2021-02-09,2021-02-16,23:59,level_2k,DSA,6.318681,s/o Gopal Vikneswaran
3,DSA2101,2020.0,Take-Home Midterms,30.000000,Assignment,I,2021-03-02,2021-03-06,23:59,level_2k,DSA,6.923077,s/o Gopal Vikneswaran
4,DSA2101,2020.0,Assignment 3,2.500000,Assignment,I,2022-09-28,2021-03-23,23:59,level_2k,DSA,6.318681,s/o Gopal Vikneswaran
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,ST4253,2210.0,Participation,10.000000,Participation,I,2022-09-28,2023-04-14,23:59,level_4k,ST,4.065934,Jialiang Li
116,ST4253,2210.0,Tutorial Presentation,10.000000,Presentation,I,2022-09-28,2023-04-14,23:59,level_4k,ST,4.395604,Jialiang Li
117,ST4248,2220.0,Presentation,10.000000,Presentation,G,2022-09-28,2023-04-14,23:59,level_4k,ST,4.835165,Ching Hway Lim
118,ST3247,2220.0,Tutorial Attendance,10.000000,Participation,I,2022-09-28,2023-04-14,23:59,level_3k,ST,3.846154,Ching Hway Lim


### Given an instructor name and date, get a df with a stress score for that day

In [18]:
# convert a date to its semester code
# to filter out assignments with incorrect due date (due date not in the semester)
def date_to_sem(date):
    year = date.year
    if date.month < 6:
        sem = str(year-1)[-2:] + '20'
    else:
        sem = str(year)[-2:] + '10'
    return int(sem)

In [19]:
# input = { instructor_scores : indiv_scores with additional instructor column,
#                 instructor :  the instructor logged in,
#                 date       :  date to calculate score for }
# output =  { daily_stress : stress score for that day, to be displayed to the given instructor }
def daily_stress_score(instructor_scores, instructor, date):
    
    # Filter by date
    df = instructor_scores[instructor_scores['Due Date'] == date]
    
    # Check that semester is correct
    sem = date_to_sem(date)
    df = df[df['Semester'] == sem]
    
    # if no assignments on that day
    if df.empty:
        return 0
    
    # if there are assignments on that day
    else:
        # check for clashes
        if clash_pairs(df, date) is None:
            return df['Stress'].tolist()[0]

        # if clashes
        else:
            # Get all clash pairs
            df1 = clash_pairs(df, date)

            # Filter clash pairs by instructor to get assignments affected for the instructor
            module_list = list(instructor_scores[instructor_scores['Instructor'] == instructor]['Module Code'].unique())
            df1 = df1[(df1['Module 1'].isin(module_list)) | (df1['Module 2'].isin(module_list))]

            # Get pairwise scores
            df1 = df1.merge(pair_counts, how='left', on=['Semester', 'Module 1', 'Module 2'])

            df1 = pd.merge(df1, student_counts, left_on=['Semester', 'Module 1'], right_on=['sem', 'code'], how = 'left')

            df1 = pd.merge(df1, student_counts, left_on=['Semester', 'Module 2'], right_on=['sem', 'code'], how = 'left')


            df1.rename(columns={"n_x":"Module 1 Count", "n_y":"Module 2 Count", "Count":"Pair Count"}, inplace=True)
            df1.drop(["sem_x", "sem_y", "code_x", "code_y"], axis=1, inplace=True)

            daily_score = 0
            for i in range(0, len(df1)):
                pair_cnt = df1.loc[i, "Pair Count"]
                m1_cnt = df1.loc[i, "Module 1 Count"]
                m2_cnt = df1.loc[i, "Module 2 Count"]
                stress_sum = sum(df1.loc[i, "Stress"])

                pairwise_score = get_pairwise_stress(pair_cnt, m1_cnt, m2_cnt, stress_sum) 
                daily_score += pairwise_score

            return daily_score

In [20]:
daily_stress_score(instructor_scores, 'Ching Hway Lim', pd.to_datetime('2023-02-07'))

4.45054945054945

### A range of days

In [21]:
def date_range(start_date, end_date):

    days = []
    current_date = start_date

    while current_date <= end_date:
        days.append(current_date)
        current_date += timedelta(days=1)
    
    return days

In [22]:
date_range(pd.to_datetime('2023-02-06'), pd.to_datetime('2023-02-14'))

[Timestamp('2023-02-06 00:00:00'),
 Timestamp('2023-02-07 00:00:00'),
 Timestamp('2023-02-08 00:00:00'),
 Timestamp('2023-02-09 00:00:00'),
 Timestamp('2023-02-10 00:00:00'),
 Timestamp('2023-02-11 00:00:00'),
 Timestamp('2023-02-12 00:00:00'),
 Timestamp('2023-02-13 00:00:00'),
 Timestamp('2023-02-14 00:00:00')]

In [23]:
def multiple_days(instructor_scores, instructor, start_date, end_date):
    days = date_range(start_date, end_date)
    
    data = []
    for d in days:
        date = d
        daily_score = daily_stress_score(instructor_scores, instructor, d)
        data.append({'Date': date, 'Daily Stress Score': daily_score})
    
    daily_scores_df = pd.DataFrame(data)
    return daily_scores_df

In [24]:
multiple_days(instructor_scores, 'Ching Hway Lim', pd.to_datetime('2023-03-01'), pd.to_datetime('2023-03-31'))

,Date,Daily Stress Score
0,2023-03-01,0.000000
1,2023-03-02,0.000000
2,2023-03-03,0.000000
3,2023-03-04,0.000000
4,2023-03-05,0.000000
5,2023-03-06,7.951719
6,2023-03-07,0.000000
7,2023-03-08,0.000000
8,2023-03-09,4.615385
9,2023-03-10,0.000000
